In [2]:
# Imports
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
# Use this function to grab the column headers from the tables on the various PGA Tour stats pages
def get_headers(soup):
    
    headers = []
    
    # Gets the Rounds header and append to the list (the Rounds headers are in a different class than other headers)
    rounds = soup.find_all(class_="rounds hidden-small hidden-medium")[0].get_text()
    headers.append(rounds)
    
    # Gets the other headers and appends them to the list as well
    stat_headers = soup.find_all(class_="col-stat hidden-small hidden-medium")
    for header in stat_headers:
        headers.append(header.get_text())
    
    return headers

In [4]:
# Get the player names from the html on the PGA Tour website
def get_players(soup):
    
    player_list = []
    
    # Get the names of the players which are located in the <a> tag right after the href link,
    # which is within the <td> tag
    players = soup.select('td a')[1:]
    # Loop through the players and append all the player names to the player_list
    for player in players:
        player_list.append(player.get_text())
    
    return player_list

In [5]:
# Get the stats from a specified url page and also input the number of stats columns in the table 
def get_stats(soup, categories):
    
    # Gets all the stats contained within the table data
    stats = soup.find_all(class_="hidden-small hidden-medium")
    
    stat_list = []
    
    # Loop through the stats table on the web page, loop though each column (category), pull the text from the table, 
    # and then append the data to the stat_list
    for i in range(0, len(stats)-categories+1, categories):
        temp_list = []
        for j in range(categories):
            temp_list.append(stats[i + j].get_text())
        stat_list.append(temp_list)
            
    return stat_list

In [6]:
# Grab the players list and stats list from the previous to functions and place them in a dictionary
# where the player name is the key and the stats are the values
def stats_dict(players, stats):
    
    player_dict = {}
    
    # Loop through the player list
    for i, player in enumerate(players):
        player_dict[player] = stats[i]
    
    return player_dict

In [7]:
# This function takes the other four functions and applies their use to a specific url and 
# then creates a dataframe using all the info that has been scraped from the url page
def create_df(url, categories):
        
    # Create BeautifulSoup object; parse with 'html.parser'
    response = requests.get(url)
    text = response.text
    soup = BeautifulSoup(text, 'lxml')
    
    # Get the column headers from the url
    headers = get_headers(soup)
    
    # Get the player names from the url
    players = get_players(soup)
    
    # Get the stats from the url
    stats = get_stats(soup, categories)
    
    # Make the stats dictionary
    stats_dictionary = stats_dict(players, stats)
    
    # Make the dataframe (the '.T' transposes all the dataframes to have the same dtype which is object)
    df = pd.DataFrame(stats_dictionary, index = headers).T
    
    # Reset the index
    df = df.reset_index()
    
    # Change the index column to 'NAME' to reference player name
    df = df.rename(index = str, columns = {'index': 'NAME'})

    return df

In [8]:
# Scrape all of the urls that have the stats we need
# Fedex cup points, top 10's, and wins
fedex_points = create_df("https://www.pgatour.com/stats/stat.02394.html", 6)\
                        [['NAME', 'POINTS', "# OF TOP 10'S", '# OF WINS']]

# Total rounds played and average score
scoring = create_df("https://www.pgatour.com/stats/stat.120.html", 5)[['NAME', 'ROUNDS', 'AVG']]
scoring = scoring.rename(columns={'AVG':'Avg Score'})

# Driving Distance
drive_distance = create_df("https://www.pgatour.com/stats/stat.101.html", 4)[['NAME', 'AVG.']]
# Rename columns
drive_distance = drive_distance.rename(columns={'AVG.':'DRIVE_DISTANCE'})

# Fairway Percentage
fairway_percent = create_df("https://www.pgatour.com/stats/stat.102.html", 4)[['NAME', '%']]
# Change column name
fairway_percent = fairway_percent.rename(columns={'%': "Fairway_%"})

# GIR (Greens in Regulation)
gir = create_df("https://www.pgatour.com/stats/stat.103.html", 5)[['NAME', '%']]
# Change column name from % to GIR %
gir = gir.rename(columns={'%': "GIR_%"})

# Money leaders
money = create_df("https://www.pgatour.com/stats/stat.109.html", 3)[['NAME', 'MONEY']]

# Average strokes gained from putting
average_SG_putt = create_df("https://www.pgatour.com/stats/stat.02564.html", 4)[['NAME', 'AVERAGE']]
average_SG_putt = average_SG_putt.rename(columns={'AVERAGE': 'Avg SG Putts'})
average_SG_putt

# Scrambling (% of time a player misses the green in regulation, but still scores a par or better)
scrambling_percent = create_df("https://www.pgatour.com/stats/stat.130.html", 4)[['NAME', '%']]
scrambling_percent = scrambling_percent.rename(columns={'%': 'Scrambling %'})

# Average SG Total (per round average of the number of strokes a player was better or worse than the field average)
average_SG_total = create_df("https://www.pgatour.com/stats/stat.02675.html", 6)[['NAME', 'AVERAGE']]
average_SG_total = average_SG_total.rename(columns={'AVERAGE': 'Avg SG Total'})

# Average SG Off the Tee (average strokes gained or lost by a player off the tee on Par 4's and 5's)
average_SG_OTT = create_df("https://www.pgatour.com/stats/stat.02567.html", 4)[['NAME', 'AVERAGE']]
average_SG_OTT = average_SG_OTT.rename(columns={'AVERAGE': 'Avg SG OTT'})

# Average SG Approach (average strokes gained or lost on shots all shots approaching the green that are not 
# drives off the tee on Par 4's and 5's)
average_SG_APP = create_df("https://www.pgatour.com/stats/stat.02568.html", 4)[['NAME', 'AVERAGE']]
average_SG_APP = average_SG_APP.rename(columns={'AVERAGE': 'Avg SG APP'})

In [9]:
# Merge all the dataframes together (fedex_points and scoring)
merged_df = pd.merge(fedex_points, scoring, how='outer', on='NAME')
# Merge drive_distance df
merged_df = pd.merge(merged_df, drive_distance, how='outer', on='NAME')
# Merge the fairway_percent df
merged_df = pd.merge(merged_df, fairway_percent, how='outer', on='NAME')
# Merge the greens in regulation df
merged_df = pd.merge(merged_df, gir, how='outer', on='NAME')
# Merge the money df
merged_df = pd.merge(merged_df, money, how='outer', on='NAME')
# Merge the scrambling_percent df
merged_df = pd.merge(merged_df, scrambling_percent, how='outer', on='NAME')
# Merge the average_SG_putt df
merged_df = pd.merge(merged_df, average_SG_putt, how='outer', on='NAME')
# Merge the average_SG_total df
merged_df = pd.merge(merged_df, average_SG_total, how='outer', on='NAME')
# Merge the average_SG_OTT df
merged_df = pd.merge(merged_df, average_SG_OTT, how='outer', on='NAME')
# Merge the average_SG_APP df
merged_df = pd.merge(merged_df, average_SG_APP, how='outer', on='NAME')

# Drop all players who have any NaN values
merged_df = merged_df.dropna(how='any') 

# Add 2019 year column
merged_df['Year'] = 2019

In [10]:
# Rearrange the columns in a different order
merged_df = merged_df[["NAME", "Year", "ROUNDS", "Avg Score", "# OF WINS", "# OF TOP 10'S",\
                       "POINTS", "Fairway_%", "DRIVE_DISTANCE", "GIR_%", "Scrambling %",\
                       "Avg SG Putts", "Avg SG OTT", "Avg SG APP", "Avg SG Total", "MONEY"]]
merged_df.head()

,NAME,Year,ROUNDS,Avg Score,# OF WINS,# OF TOP 10'S,POINTS,Fairway_%,DRIVE_DISTANCE,GIR_%,Scrambling %,Avg SG Putts,Avg SG OTT,Avg SG APP,Avg SG Total,MONEY
0,Brooks Koepka,2019,77,69.448,3,8,"2,887",62.50,308.3,70.08,62.07,.210,.527,.595,1.353,"$9,684,006"
1,Rory McIlroy,2019,68,69.203,2,12,"2,315",61.67,314.0,68.49,63.09,.416,1.185,.621,2.495,"$7,785,286"
2,Matt Kuchar,2019,83,69.874,2,8,"2,313",67.95,289.5,70.10,63.33,.252,.100,.710,1.177,"$6,294,690"
3,Xander Schauffele,2019,73,69.969,2,5,"1,858",60.77,303.7,68.25,62.00,.231,.485,.356,1.300,"$5,609,456"
4,Gary Woodland,2019,83,70.230,1,8,"1,795",63.81,308.7,69.24,57.56,-.113,.527,.564,.954,"$5,690,965"


In [11]:
# Read in the pga tour csv with data from 2010 to 2018
pga_tour_csv = "Resources/PGA_Tour_Data_2010_2018.csv"
pga_data = pd.read_csv(pga_tour_csv, delimiter=',', encoding='latin-1')
pga_data_df = pd.DataFrame(pga_data)

In [12]:
# Fill all NaN values with ' ' to match our other dataframe
pga_data_df = pga_data_df.fillna('0')

# Drop the 2 columns that we do not need (Average Putts and SG:ARG)
# pga_data_df = pga_data_df.drop(columns={'Average Putts', 'SG:ARG'})

In [13]:
# Rearrange the columns of pga_data_df to match the order of merged_df
pga_data_df = pga_data_df[["Player Name", "Year", "Rounds", "Average Score", "Wins", "Top 10",\
                           "Points", "Fairway Percentage", "Avg Distance", "gir",\
                           "Average Scrambling", "Average SG Putts", "SG:OTT", "SG:APR",\
                           "Average SG Total", "Money"]]
pga_data_df

,Player Name,Year,Rounds,Average Score,Wins,Top 10,Points,Fairway Percentage,Avg Distance,gir,Average Scrambling,Average SG Putts,SG:OTT,SG:APR,Average SG Total,Money
0,Henrik Stenson,2018,60,69.617,0,5,868,75.19,291.5,73.51,60.67,-0.207,0.427,0.960,1.153,"$2,680,487"
1,Ryan Armour,2018,109,70.758,1,3,"1,006",73.58,283.5,68.22,60.13,-0.058,-0.012,0.213,0.337,"$2,485,203"
2,Chez Reavie,2018,93,70.432,0,3,"1,020",72.24,286.5,68.67,62.27,0.192,0.183,0.437,0.674,"$2,700,018"
3,Ryan Moore,2018,78,70.015,0,5,795,71.94,289.2,68.80,64.16,-0.271,0.406,0.532,0.941,"$1,986,608"
4,Brian Stuard,2018,103,71.038,0,3,421,71.44,278.9,67.12,59.23,0.164,-0.227,0.099,0.062,"$1,089,763"
5,Brian Gay,2018,103,70.280,0,6,880,71.37,282.9,64.52,63.26,0.442,-0.166,0.036,0.565,"$2,152,501"
6,Kyle Stanley,2018,93,70.404,0,5,"1,198",71.29,295.7,71.09,54.80,0.037,0.378,0.298,0.686,"$3,916,001"
7,Emiliano Grillo,2018,94,70.152,0,5,901,70.16,295.2,68.84,61.05,0.546,0.364,0.345,1.133,"$2,493,163"
8,Russell Henley,2018,77,70.489,0,3,569,70.03,293.0,68.77,54.33,0.167,0.093,0.467,0.541,"$1,516,438"
9,Jim Furyk,2018,50,70.342,0,2,291,69.91,280.5,63.19,62.58,0.389,-0.392,0.179,0.412,"$660,010"


In [14]:
# Rename the columns of our pga_data_df dataframe
pga_data_df = pga_data_df.rename(columns={
    "Player Name": "Player_Name",
    "Average Score": "Avg_Score",
    "Top 10": "Top_10",
    "Fairway Percentage": "Fairway_Pct",
    "Avg Distance": "Avg_Drive_Distance",
    "gir": "GIR_Pct",
    "Average Scrambling": "Scrambling_Pct",
    "Average SG Putts": "Avg_SG_Putts",
    "SG:OTT": "Avg_SG_OTT",
    "SG:APR": "Avg_SG_APP",
    "Average SG Total": "Avg_SG_Total",
    "Money": "Earnings"
})

# Remove the commas from the Points column
pga_data_df["Points"] = pga_data_df["Points"].str.replace(",","").astype(float)

pga_data_df.head()

,Player_Name,Year,Rounds,Avg_Score,Wins,Top_10,Points,Fairway_Pct,Avg_Drive_Distance,GIR_Pct,Scrambling_Pct,Avg_SG_Putts,Avg_SG_OTT,Avg_SG_APP,Avg_SG_Total,Earnings
0,Henrik Stenson,2018,60,69.617,0,5,868.0,75.19,291.5,73.51,60.67,-0.207,0.427,0.960,1.153,"$2,680,487"
1,Ryan Armour,2018,109,70.758,1,3,1006.0,73.58,283.5,68.22,60.13,-0.058,-0.012,0.213,0.337,"$2,485,203"
2,Chez Reavie,2018,93,70.432,0,3,1020.0,72.24,286.5,68.67,62.27,0.192,0.183,0.437,0.674,"$2,700,018"
3,Ryan Moore,2018,78,70.015,0,5,795.0,71.94,289.2,68.80,64.16,-0.271,0.406,0.532,0.941,"$1,986,608"
4,Brian Stuard,2018,103,71.038,0,3,421.0,71.44,278.9,67.12,59.23,0.164,-0.227,0.099,0.062,"$1,089,763"


In [15]:
# Round the decimals to 3 places in these columns
Cols = ["Avg_Score", "Avg_SG_Putts", "Avg_SG_OTT", "Avg_SG_APP", "Avg_SG_Total"]
pga_data_df[Cols] = pga_data_df[Cols].round(3)

In [16]:
# Rename the columns of our merged_df dataframe to match the columns in our pga_data_df dataframe
merged_df = merged_df.rename(columns={
    "NAME": "Player_Name",
    "ROUNDS": "Rounds",
    "Avg Score": "Avg_Score",
    "POINTS": "Points",
    "# OF TOP 10'S": "Top_10",
    "# OF WINS": "Wins",
    "DRIVE_DISTANCE": "Avg_Drive_Distance",
    "Fairway_%": "Fairway_Pct",
    "GIR_%": "GIR_Pct",
    "Scrambling %": "Scrambling_Pct",
    "Avg SG Putts": "Avg_SG_Putts",
    "Avg SG OTT": "Avg_SG_OTT",
    "Avg SG APP": "Avg_SG_APP",
    "Avg SG Total": "Avg_SG_Total",
    "MONEY": "Earnings"
})

# Remove the commas from the Points column
merged_df["Points"] = merged_df["Points"].str.replace(",","").astype(float)

merged_df.head()

,Player_Name,Year,Rounds,Avg_Score,Wins,Top_10,Points,Fairway_Pct,Avg_Drive_Distance,GIR_Pct,Scrambling_Pct,Avg_SG_Putts,Avg_SG_OTT,Avg_SG_APP,Avg_SG_Total,Earnings
0,Brooks Koepka,2019,77,69.448,3,8,2887.0,62.50,308.3,70.08,62.07,.210,.527,.595,1.353,"$9,684,006"
1,Rory McIlroy,2019,68,69.203,2,12,2315.0,61.67,314.0,68.49,63.09,.416,1.185,.621,2.495,"$7,785,286"
2,Matt Kuchar,2019,83,69.874,2,8,2313.0,67.95,289.5,70.10,63.33,.252,.100,.710,1.177,"$6,294,690"
3,Xander Schauffele,2019,73,69.969,2,5,1858.0,60.77,303.7,68.25,62.00,.231,.485,.356,1.300,"$5,609,456"
4,Gary Woodland,2019,83,70.230,1,8,1795.0,63.81,308.7,69.24,57.56,-.113,.527,.564,.954,"$5,690,965"


In [17]:
# Round the decimals to 3 places in these columns
Cols = ["Avg_Score", "Avg_SG_Putts", "Avg_SG_OTT", "Avg_SG_APP", "Avg_SG_Total"]
merged_df[Cols] = merged_df[Cols].round(3)

In [18]:
# Save the two dataframes as csv files to be imported into postgres
pga_data_df.to_csv("Resources/PGA_2010_2018.csv")
merged_df.to_csv("Resources/PGA_2019.csv")